In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd
import re

URL = "https://www.teamblind.com/kr/company/%ED%98%84%EB%8C%80%EC%9E%90%EB%8F%99%EC%B0%A8/reviews"
lastPage =25

driver = webdriver.Chrome(service= Service(ChromeDriverManager().install()))
driver.get(URL)
driver.find_element(By.CLASS_NAME,"btn_signin").click()
time.sleep(30)

#컨텐츠 부분 특수문제 제거
def clean_str(text):
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '<[^>]*>'         # HTML 태그 제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '[^\w\s]'         # 특수기호제거
    text = re.sub(pattern=pattern, repl='', string=text)
    text = text.replace('\r','. ')
    return text

result = []

colname = ["총점","커리어 향상","업무와 삶의 균형","급여 및 복지","사내 문화", "경영진", "제목","직원유형","장점","단점"]

for p in range(1,40):
    targetURL = URL +"?page="+ str(p)
    driver.get(targetURL)
    elements = driver.find_elements(By.CLASS_NAME,"review_item")
    time.sleep(5)

    for e in elements:
        rate = e.find_element(By.CLASS_NAME,"rating")
        
        rate.find_element(By.CLASS_NAME,"more_rating").click()
        score = rate.find_element(By.CLASS_NAME,"num").text
        score = float(score.split("\n")[1])
        
        detailScores = e.find_elements(By.CSS_SELECTOR,"div.review_item_inr > div.rating > div.more_rating > span > span > span > div.ly_rating > div.rating_wp> span.desc > i.blind")
        title = e.find_element(By.CSS_SELECTOR,"div.review_item_inr > h3.rvtit > a").text
        status = e.find_element(By.CSS_SELECTOR,"div.review_item_inr > div.auth").text
        status = status.split("\n")[1]
        
        prosCons = e.find_elements(By.CSS_SELECTOR,"div.review_item_inr > div > div.parag > p > span")
        value = [score, detailScores[0].text, detailScores[1].text, detailScores[2].text, detailScores[3].text,
                 detailScores[4].text, title, status, clean_str(prosCons[0].text), clean_str(prosCons[1].text)]
        result.append(value)
        print(value)

df = pd.DataFrame(result,columns=colname)
df.head()